In [1]:
# ==============================================================================
# Cell 1: Introduction, Setup, and Configuration (Updated)
# ==============================================================================
#
# NEW FEATURES in this version:
# - Stochastic Weight Averaging (SWA) for the Generator to find a better,
#   more stable minimum in the loss landscape.
# - Detailed, normalized Fréchet Distance reporting, split by feature type.
# - A more robust implementation of the mode-seeking diversity loss.
#
# ==============================================================================

# --- 1.1. Install necessary libraries ---
!pip install  numpy scipy scikit-learn matplotlib seaborn pandas mne --quiet

# --- 1.2. Imports ---
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.swa_utils import AveragedModel, SWALR
import matplotlib.pyplot as plt
from scipy import signal
from scipy.linalg import sqrtm
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# --- 1.3. Configuration & Setup ---
OUTPUT_DIR = "/kaggle/working/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model and Data Parameters
DATA_PATH = "/kaggle/input/mpi-lemon-eeg/lemon_preprocessed_8ch_512ts.npy"
CHANNELS = 8
TIMESTEPS = 512
LATENT_DIM = 100
NUM_CLASSES = 2

# Training Hyperparameters
NUM_EPOCHS = 300 # Increased for longer training
BATCH_SIZE = 64
LR_G = 1e-4
LR_D = 1e-4
CRITIC_ITERATIONS = 5
DIVERSITY_WEIGHT = 0.5  # A slightly stronger weight for diversity
LAMBDA_GP_FINAL = 10.0
ANNEAL_EPOCHS = 50
SAVE_INTERVAL = 50  # Added missing SAVE_INTERVAL

# --- NEW: SWA Configuration ---
SWA_START_EPOCH = 240 # Start SWA in the last 100 epochs
SWA_LR = 5e-5          # A smaller learning rate for the SWA phase

# Setup device
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"SWA will start at epoch {SWA_START_EPOCH}")


Using device: cuda
SWA will start at epoch 240


In [2]:
# ==============================================================================
# Cell 2: Load Preprocessed Data
# ==============================================================================
# This cell is unchanged. It loads your .npy file.
# ==============================================================================

try:
    # IMPORTANT: Upload your 'lemon_preprocessed_8ch_512ts.npy' file to a Kaggle
    # dataset and add it to this notebook. Then, update the path in Cell 1.
    data = np.load(DATA_PATH)
    print(f"Successfully loaded data. Shape: {data.shape}")
    labels = np.random.randint(0, NUM_CLASSES, data.shape[0])
    print(f"Created dummy labels. Shape: {labels.shape}")
    
    dataset = TensorDataset(torch.from_numpy(data).float(), torch.from_numpy(labels).long())
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    print("Created DataLoader successfully.")
    
except FileNotFoundError:
    print(f"ERROR: Data file not found at '{DATA_PATH}'.")
    print("Please upload your preprocessed .npy file as a Kaggle Dataset and update the path in Cell 1.")
except Exception as e:
    print(f"An error occurred: {e}")


Successfully loaded data. Shape: (35503, 512, 8)
Created dummy labels. Shape: (35503,)
Created DataLoader successfully.


In [3]:
# ==============================================================================
# Cell 3: Model Architectures (with `.reshape()`)
# ==============================================================================
# Using .reshape() instead of .view() for clarity and safety.
# ==============================================================================

from torch.nn.utils import spectral_norm

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.BatchNorm1d(in_channels),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
        )
        self.shortcut = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0)
        )
    
    def forward(self, x):
        return self.conv_block(x) + self.shortcut(x)

class ResGenerator(nn.Module):
    def __init__(self, latent_dim, num_classes, channels, timesteps):
        super(ResGenerator, self).__init__()
        self.initial_len = timesteps // 16
        self.label_emb = nn.Embedding(num_classes, latent_dim)
        self.fc = nn.Linear(latent_dim * 2, 256 * self.initial_len)
        
        self.res_blocks = nn.Sequential(
            ResBlock(256, 128), 
            ResBlock(128, 64),
            ResBlock(64, 32), 
            ResBlock(32, 16),
        )
        
        self.final_conv = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(16, channels, kernel_size=3, padding=1),
            nn.Tanh()
        )
    
    def forward(self, noise, labels):
        c = self.label_emb(labels)
        x = torch.cat([noise, c], 1)
        x = self.fc(x)
        x = x.reshape(x.size(0), 256, self.initial_len)
        x = self.res_blocks(x)
        x = self.final_conv(x)
        return x.transpose(1, 2)

class ImprovedDiscriminator(nn.Module):
    def __init__(self, num_classes, channels, timesteps):
        super(ImprovedDiscriminator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, 50)
        
        self.conv_layers = nn.Sequential(
            spectral_norm(nn.Conv1d(channels, 64, 5, 2, 2)), 
            nn.LeakyReLU(0.2, inplace=True),
            spectral_norm(nn.Conv1d(64, 128, 5, 2, 2)), 
            nn.LeakyReLU(0.2, inplace=True),
            spectral_norm(nn.Conv1d(128, 256, 5, 2, 2)), 
            nn.LeakyReLU(0.2, inplace=True),
            spectral_norm(nn.Conv1d(256, 512, 5, 2, 2)), 
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        # Calculate conv output size
        with torch.no_grad():
            dummy_input = torch.randn(1, channels, timesteps)
            conv_out_size = self.conv_layers(dummy_input).reshape(1, -1).size(1)
        
        self.classifier = nn.Sequential(
            spectral_norm(nn.Linear(conv_out_size + 50, 1024)), 
            nn.LeakyReLU(0.2, inplace=True), 
            nn.Dropout(0.5),
            spectral_norm(nn.Linear(1024, 512)), 
            nn.LeakyReLU(0.2, inplace=True), 
            nn.Dropout(0.5),
            spectral_norm(nn.Linear(512, 1))
        )
    
    def forward(self, eeg, labels):
        if self.training:
            noise = torch.randn_like(eeg) * 0.05
            eeg = eeg + noise
        
        x = eeg.transpose(1, 2)
        x = self.conv_layers(x)
        x = x.reshape(x.size(0), -1)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        return self.classifier(x)

print("GAN model architectures defined.")

GAN model architectures defined.


In [4]:
# ==============================================================================
# Cell 4: The Complete YARE-GAN Evaluation & Visualization Suite
# ==============================================================================
#
# This cell contains all functions to generate the detailed metrics and plots
# required to benchmark against the YARE-GAN paper (Figures 3, 4, 5, 6).
#
# ==============================================================================
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import skew, kurtosis
import matplotlib.animation as animation
from IPython.display import HTML

# --- 1. DETAILED FEATURE EXTRACTION ---

def get_detailed_features(data, sfreq=98):
    """
    Extracts detailed time-domain and frequency-domain features for evaluation.
    Input data shape: (n_epochs, n_timesteps, n_channels)
    """
    data_t = data.transpose(0, 2, 1) # -> (n_epochs, n_channels, n_timesteps)

    # --- Frequency-Domain Features ---
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 45)}
    freqs, psd = signal.welch(data_t, fs=sfreq, axis=2)
    abs_band_powers = np.stack([np.mean(psd[:, :, np.logical_and(freqs >= b[0], freqs <= b[1])], axis=2) for b in bands.values()], axis=-1)
    total_power = np.sum(abs_band_powers, axis=-1, keepdims=True)
    relative_psd_features = abs_band_powers / (total_power + 1e-10)

    # --- Time-Domain Features ---
    dx, ddx = np.diff(data_t, axis=2), np.diff(np.diff(data_t, axis=2), axis=2)
    var_x, var_dx = np.var(data_t, axis=2), np.var(dx, axis=2)
    activity = var_x
    mobility = np.sqrt(var_dx / (var_x + 1e-8))
    complexity = np.sqrt(np.var(ddx, axis=2) / (var_dx + 1e-8)) / (mobility + 1e-8)
    hjorth_features = np.stack([activity, mobility, complexity], axis=-1)
    
    skewness_features = skew(data_t, axis=2)[:, :, np.newaxis]
    kurtosis_features = kurtosis(data_t, axis=2)[:, :, np.newaxis]
    
    # --- Prepare for FD calculation (flatten features) ---
    spectral_flat = relative_psd_features.reshape(data.shape[0], -1)
    time_domain_flat = np.concatenate([
        hjorth_features.reshape(data.shape[0], -1),
        skewness_features.reshape(data.shape[0], -1),
        kurtosis_features.reshape(data.shape[0], -1)
    ], axis=1)
    
    return spectral_flat, time_domain_flat

def calculate_frechet_distance(features1, features2):
    mu1, sigma1 = np.mean(features1, axis=0), np.cov(features1, rowvar=False)
    mu2, sigma2 = np.mean(features2, axis=0), np.cov(features2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean): covmean = covmean.real
    fd = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fd

def get_final_fd_scores(real_data, gen_data, n_channels):
    """Calculates and returns the final set of FD scores, normalized and unnormalized."""
    real_spectral, real_time = get_detailed_features(real_data)
    gen_spectral, gen_time = get_detailed_features(gen_data)

    fd_spectral = calculate_frechet_distance(real_spectral, gen_spectral)
    fd_time = calculate_frechet_distance(real_time, gen_time)
    
    return {
        "FD Spectral (Normalized)": fd_spectral / n_channels,
        "FD Time (Normalized)": fd_time / n_channels, # YARE-GAN used Hjorth, we use a more comprehensive time metric
        "FD Total": fd_spectral + fd_time
    }

# --- 2. VISUALIZATION FUNCTIONS (The YARE-GAN Playbook) ---

def plot_distributions(real_data, gen_data, save_path_prefix):
    """Generates plots for Figure 3a and 3b from the YARE-GAN paper."""
    # Figure 3a: PCA Plot
    pca = PCA(n_components=2)
    real_flat, gen_flat = real_data.reshape(real_data.shape[0], -1), gen_data.reshape(gen_data.shape[0], -1)
    pca.fit(real_flat)
    real_pca, gen_pca = pca.transform(real_flat), pca.transform(gen_flat)
    
    plt.figure(figsize=(8, 8));
    plt.scatter(real_pca[:, 0], real_pca[:, 1], alpha=0.5, label='Real EEG', s=10, c='blue')
    plt.scatter(gen_pca[:, 0], gen_pca[:, 1], alpha=0.5, label='Generated EEG', s=10, c='orange')
    plt.title('PCA Distribution of Real vs. Generated Data', fontsize=16); plt.xlabel('PC 1'); plt.ylabel('PC 2')
    plt.legend(); plt.grid(True, linestyle='--', alpha=0.6); plt.savefig(f"{save_path_prefix}_fig3a_pca.png"); plt.close()

    # Figure 3b: Global Histogram
    plt.figure(figsize=(10, 6))
    sns.histplot(real_data.flatten(), color="blue", label='Real', stat='density', bins=100, alpha=0.6)
    sns.histplot(gen_data.flatten(), color="orange", label='Generated', stat='density', bins=100, alpha=0.6)
    plt.title('Global Amplitude Distribution', fontsize=16); plt.xlabel('Amplitude (Normalized)'); plt.ylabel('Density')
    plt.legend(); plt.savefig(f"{save_path_prefix}_fig3b_histogram.png"); plt.close()
    print("Generated distribution plots (Fig 3a, 3b).")

def plot_psd_per_channel(real_data, gen_data, save_path_prefix):
    """Generates plots for Figure 4 from the YARE-GAN paper."""
    channel_names = ['F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'O1', 'O2']; sfreq = 98
    fig, axes = plt.subplots(4, 2, figsize=(15, 20), sharex=True, sharey=True); axes = axes.flatten()

    for i in range(real_data.shape[2]):
        f_real, psd_real = signal.welch(real_data[:, :, i], fs=sfreq, axis=1)
        f_gen, psd_gen = signal.welch(gen_data[:, :, i], fs=sfreq, axis=1)
        
        mean_psd_real, std_psd_real = np.mean(np.log10(psd_real + 1e-12), 0), np.std(np.log10(psd_real + 1e-12), 0)
        mean_psd_gen, std_psd_gen = np.mean(np.log10(psd_gen + 1e-12), 0), np.std(np.log10(psd_gen + 1e-12), 0)

        ax = axes[i]
        ax.plot(f_real, mean_psd_real, label='Real', color='blue'); ax.fill_between(f_real, mean_psd_real - std_psd_real, mean_psd_real + std_psd_real, color='blue', alpha=0.2)
        ax.plot(f_gen, mean_psd_gen, label='Generated', color='orange'); ax.fill_between(f_gen, mean_psd_gen - std_psd_gen, mean_psd_gen + std_psd_gen, color='orange', alpha=0.2)
        ax.set_title(f'Channel: {channel_names[i]}'); ax.set_xlabel('Frequency (Hz)'); ax.set_ylabel('Log Power/Hz')
        ax.legend(); ax.grid(True, linestyle='--')

    plt.tight_layout(); fig.suptitle('Power Spectral Density Comparison per Channel', fontsize=20, y=1.02)
    plt.savefig(f"{save_path_prefix}_fig4_psd_per_channel.png"); plt.close()
    print("Generated per-channel PSD plots (Fig 4).")

def plot_connectivity_matrix(real_data, gen_data, save_path_prefix):
    """Generates plots for Figure 5 from the YARE-GAN paper."""
    real_flat, gen_flat = real_data.reshape(-1, real_data.shape[2]), gen_data.reshape(-1, gen_data.shape[2])
    conn_real, conn_gen = cosine_similarity(real_flat.T), cosine_similarity(gen_flat.T)
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    sns.heatmap(conn_real, ax=axes[0], annot=True, fmt='.2f', cmap='viridis', vmin=0, vmax=1); axes[0].set_title('Real EEG Connectivity', fontsize=14)
    sns.heatmap(conn_gen, ax=axes[1], annot=True, fmt='.2f', cmap='viridis', vmin=0, vmax=1); axes[1].set_title('Generated EEG Connectivity', fontsize=14)
    fig.suptitle('Functional Connectivity Comparison', fontsize=18); plt.savefig(f"{save_path_prefix}_fig5_connectivity.png"); plt.close()
    print("Generated connectivity plots (Fig 5).")

def plot_fd_progression(frechet_distances, save_interval, save_path):
    """Generates plots for Figure 6 from the YARE-GAN paper."""
    fig, axes = plt.subplots(1, 2, figsize=(18, 6)); fd_epochs = np.arange(1, len(frechet_distances) + 1) * save_interval
    fd_spectral = [d['FD Spectral (Normalized)'] for d in frechet_distances]
    fd_time = [d['FD Time (Normalized)'] for d in frechet_distances]
    axes[0].plot(fd_epochs, fd_spectral, 'b-o', label='Spectral FD'); axes[0].set_title('Spectral FD Progression'); axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('FD (Normalized)'); axes[0].legend(); axes[0].grid(True)
    axes[1].plot(fd_epochs, fd_time, 'g-o', label='Time-Domain FD'); axes[1].set_title('Time-Domain FD Progression'); axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('FD (Normalized)'); axes[1].legend(); axes[1].grid(True)
    fig.suptitle('Fréchet Distance Progression Over Training', fontsize=18); plt.savefig(save_path); plt.close()
    print("Generated FD progression plot (Fig 6).")

print("Comprehensive evaluation and plotting suite defined.")

Comprehensive evaluation and plotting suite defined.


In [5]:
import os
print(os.listdir())  # Check for torch.py or .pyc files


['__notebook__.ipynb']


In [6]:
# ==============================================================================
# Cell 5: Final Optimized Training Loop (250 Epochs) - BUG FIXED
# ==============================================================================
# - Fixes the SWA `update_bn` bug by correctly accessing the batch size from
#   the dataloader's yielded tuple: `batch_data[0].shape[0]`
# - Calls the new detailed FD function during evaluation.
# ==============================================================================

# --- Hyperparameters ---
NUM_EPOCHS = 300
SWA_START_EPOCH = 240
SCHEDULER_PATIENCE = 20

def compute_gradient_penalty(discriminator, real_samples, fake_samples, labels, device):
    alpha = torch.rand(real_samples.size(0), 1, 1, device=device); interpolates = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)
    d_interpolates = discriminator(interpolates, labels); fake = torch.ones(real_samples.size(0), 1, device=device)
    gradients = torch.autograd.grad(outputs=d_interpolates, inputs=interpolates, grad_outputs=fake, create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.reshape(gradients.size(0), -1); return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

# --- THE BUG FIX is in this function ---
def update_generator_bn(dataloader, swa_model, device, latent_dim, num_classes):
    print("Running custom BatchNorm update for SWA Generator..."); swa_model.train()
    for i, batch_data in enumerate(dataloader):
        # THE FIX: Dataloader yields a tuple (data, labels). Access the data tensor.
        batch_size = batch_data[0].shape[0]
        with torch.no_grad():
            z = torch.randn(batch_size, latent_dim, device=device); labels = torch.randint(0, num_classes, (batch_size,), device=device)
            _ = swa_model(z, labels)
        if i >= 50: break
    print("BatchNorm update complete.")

# --- Model, Optimizer, and SWA Setup ---
generator = ResGenerator(LATENT_DIM, NUM_CLASSES, CHANNELS, TIMESTEPS).to(device)
discriminator = ImprovedDiscriminator(NUM_CLASSES, CHANNELS, TIMESTEPS).to(device)
g_optimizer = optim.Adam(generator.parameters(), lr=LR_G, betas=(0.5, 0.9)); d_optimizer = optim.Adam(discriminator.parameters(), lr=LR_D, betas=(0.5, 0.9))
g_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(g_optimizer, 'min', factor=0.5, patience=SCHEDULER_PATIENCE, verbose=False)
d_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(d_optimizer, 'min', factor=0.5, patience=SCHEDULER_PATIENCE, verbose=False)
swa_model = AveragedModel(generator); swa_scheduler = SWALR(g_optimizer, swa_lr=SWA_LR)
g_losses, d_losses, frechet_distances = [], [], []; best_fd = float('inf')
real_samples_for_eval = next(iter(dataloader))[0].numpy()[:256]

print(f"Starting Optimized GAN training for {NUM_EPOCHS} epochs...");
for epoch in range(NUM_EPOCHS):
    lambda_gp = LAMBDA_GP_FINAL if epoch >= ANNEAL_EPOCHS else LAMBDA_GP_FINAL * (epoch / ANNEAL_EPOCHS)
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} (GP λ: {lambda_gp:.2f})")
    epoch_g_loss, epoch_d_loss, g_loss_val = 0, 0, torch.tensor(0.0)
    for i, (real_eegs, labels) in enumerate(pbar):
        real_eegs, labels = real_eegs.to(device), labels.to(device); batch_size = real_eegs.size(0)
        d_optimizer.zero_grad()
        with torch.no_grad(): z = torch.randn(batch_size, LATENT_DIM, device=device); fake_eegs = generator(z, labels)
        d_loss = -torch.mean(discriminator(real_eegs, labels)) + torch.mean(discriminator(fake_eegs, labels))
        gp = compute_gradient_penalty(discriminator, real_eegs.data, fake_eegs.data, labels.data, device)
        d_loss += lambda_gp * gp; d_loss.backward(); d_optimizer.step(); epoch_d_loss += d_loss.item()
        if i % CRITIC_ITERATIONS == 0:
            g_optimizer.zero_grad(); z1 = torch.randn(batch_size, LATENT_DIM, device=device); gen_labels = torch.randint(0, NUM_CLASSES, (batch_size,), device=device)
            fake_eegs1 = generator(z1, gen_labels); g_loss_adv = -torch.mean(discriminator(fake_eegs1, gen_labels))
            z2 = torch.randn(batch_size, LATENT_DIM, device=device); fake_eegs2 = generator(z2, gen_labels)
            lf = torch.mean(torch.abs(fake_eegs1 - fake_eegs2)) / (torch.mean(torch.abs(fake_eegs1)) + 1e-8)
            lz = torch.mean(torch.abs(z1 - z2)) / (torch.mean(torch.abs(z1)) + 1e-8)
            g_loss_div = - DIVERSITY_WEIGHT * (lf / (lz + 1e-8))
            g_loss = g_loss_adv + g_loss_div; g_loss.backward(); g_optimizer.step(); epoch_g_loss += g_loss.item(); g_loss_val = g_loss
        pbar.set_postfix({'D Loss': f'{d_loss.item():.4f}', 'G Loss': f'{g_loss_val.item():.4f}'})
    avg_d_loss, avg_g_loss = epoch_d_loss/len(pbar), epoch_g_loss/(len(pbar)/CRITIC_ITERATIONS)
    d_losses.append(avg_d_loss); g_losses.append(avg_g_loss)
    if epoch >= SWA_START_EPOCH: swa_model.update_parameters(generator); swa_scheduler.step()
    else: g_scheduler.step(avg_g_loss); d_scheduler.step(avg_d_loss)
    if (epoch + 1) % SAVE_INTERVAL == 0 or epoch == NUM_EPOCHS - 1:
        print(f"\n--- Epoch {epoch+1} Evaluation ---")
        print(f"  Learning Rates - G: {g_optimizer.param_groups[0]['lr']:.6f}, D: {d_optimizer.param_groups[0]['lr']:.6f}")
        generator.eval()
        with torch.no_grad():
            z_eval = torch.randn(real_samples_for_eval.shape[0], LATENT_DIM, device=device); labels_eval = torch.randint(0, NUM_CLASSES, (real_samples_for_eval.shape[0],), device=device)
            generated_samples_eval = generator(z_eval, labels_eval).cpu().numpy()
            fd_scores = get_final_fd_scores(real_samples_for_eval, generated_samples_eval, CHANNELS) # Using the new detailed function
            frechet_distances.append(fd_scores)
            print(f"  FD Scores (Norm) - Spectral: {fd_scores['FD Spectral (Normalized)']:.4f}, Time: {fd_scores['FD Time (Normalized)']:.4f}")
            if fd_scores['FD Total'] < best_fd: 
                best_fd = fd_scores['FD Total']; torch.save(generator.state_dict(), os.path.join(OUTPUT_DIR, 'generator_best.pth'));    torch.save(discriminator.state_dict(), os.path.join(OUTPUT_DIR, 'discriminator_best.pth')); print(f"  New best standard model saved with Total FD: {best_fd:.4f}")
        generator.train()
# --- Final SWA Steps ---
print("\nTraining complete. Updating SWA model batch norm statistics...")
try:
    update_generator_bn(dataloader, swa_model, device, LATENT_DIM, NUM_CLASSES)
    torch.save(swa_model.state_dict(), os.path.join(OUTPUT_DIR, 'generator_swa_best.pth')); print("SWA model saved successfully.")
except Exception as e: print(f"Warning: SWA process failed: {e}. Proceeding without SWA model.")
plot_fd_progression(frechet_distances, SAVE_INTERVAL, os.path.join(OUTPUT_DIR, 'final_fd_progression.png')) # Use the new plot
print("Final progress plot saved.")

Starting Optimized GAN training for 300 epochs...


Epoch 1/300 (GP λ: 0.00):   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 2/300 (GP λ: 0.20):   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 3/300 (GP λ: 0.40):   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 4/300 (GP λ: 0.60):   0%|          | 0/555 [00:00<?, ?it/s]

...

Epoch 297/300 (GP λ: 10.00):   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 298/300 (GP λ: 10.00):   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 299/300 (GP λ: 10.00):   0%|          | 0/555 [00:00<?, ?it/s]

Epoch 300/300 (GP λ: 10.00):   0%|          | 0/555 [00:00<?, ?it/s]


--- Epoch 300 Evaluation ---
  Learning Rates - G: 0.000050, D: 0.000000
  FD Scores (Norm) - Spectral: 0.0644, Time: 5.7367

Training complete. Updating SWA model batch norm statistics...
Running custom BatchNorm update for SWA Generator...
BatchNorm update complete.
SWA model saved successfully.
Generated FD progression plot (Fig 6).
Final progress plot saved.


In [7]:
# ==============================================================================
# Cell 6: Final Analysis and Comprehensive Visualization
# ==============================================================================
# This cell loads the best standard model and the SWA model, compares them,
# and then runs the *entire* evaluation suite from Cell 4 on the winner.
# ==============================================================================

print("\n--- Final Model Analysis & Visualization ---")
real_samples_final = next(iter(dataloader))[0].numpy()[:512]
standard_fd_scores, swa_fd_scores = None, None
generated_standard, generated_swa = None, None

# --- 1. Evaluate the BEST STANDARD model ---
try:
    print("\n--- Evaluating BEST STANDARD Model ---")
    best_standard_generator = ResGenerator(LATENT_DIM, NUM_CLASSES, CHANNELS, TIMESTEPS).to(device)
    best_standard_generator.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'generator_best.pth')))
    best_standard_generator.eval()
    with torch.no_grad():
        z_final = torch.randn(512, LATENT_DIM, device=device); labels_final = torch.randint(0, NUM_CLASSES, (512,), device=device)
        generated_standard = best_standard_generator(z_final, labels_final).cpu().numpy()
    standard_fd_scores = get_final_fd_scores(real_samples_final, generated_standard, CHANNELS)
    print(f"  > FD Scores (Norm) - Spectral: {standard_fd_scores['FD Spectral (Normalized)']:.4f}, Time: {standard_fd_scores['FD Time (Normalized)']:.4f}")
except FileNotFoundError: print("Best standard model not found.")

# --- 2. Evaluate the FINAL SWA model ---
try:
    print("\n--- Evaluating FINAL SWA Model ---")
    final_swa_model = AveragedModel(ResGenerator(LATENT_DIM, NUM_CLASSES, CHANNELS, TIMESTEPS))
    final_swa_model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'generator_swa_best.pth')))
    final_swa_model = final_swa_model.to(device); final_swa_model.eval()
    with torch.no_grad():
        z_final = torch.randn(512, LATENT_DIM, device=device); labels_final = torch.randint(0, NUM_CLASSES, (512,), device=device)
        generated_swa = final_swa_model(z_final, labels_final).cpu().numpy()
    swa_fd_scores = get_final_fd_scores(real_samples_final, generated_swa, CHANNELS)
    print(f"  > FD Scores (Norm) - Spectral: {swa_fd_scores['FD Spectral (Normalized)']:.4f}, Time: {swa_fd_scores['FD Time (Normalized)']:.4f}")
except FileNotFoundError: print("SWA model not found.")

# --- 3. Determine the WINNER and generate all plots ---
print("\n--- Generating Final Plots from the Winning Model ---")
winning_model_samples = None
if standard_fd_scores and swa_fd_scores:
    if swa_fd_scores['FD Total'] < standard_fd_scores['FD Total']: print("SWA Model is the winner!"); winning_model_samples = generated_swa
    else: print("Best Standard Model is the winner!"); winning_model_samples = generated_standard
elif standard_fd_scores: print("Only Standard model available."); winning_model_samples = generated_standard
elif swa_fd_scores: print("Only SWA model available."); winning_model_samples = generated_swa
else: print("No models available for final analysis.")

if winning_model_samples is not None:
    plot_distributions(real_samples_final, winning_model_samples, os.path.join(OUTPUT_DIR, "final_eval"))
    plot_psd_per_channel(real_samples_final, winning_model_samples, os.path.join(OUTPUT_DIR, "final_eval"))
    plot_connectivity_matrix(real_samples_final, winning_model_samples, os.path.join(OUTPUT_DIR, "final_eval"))
    print("\nAll final evaluation plots have been saved to the output directory.")
else:
    print("Skipping final plot generation as no winning model was found.")


--- Final Model Analysis & Visualization ---

--- Evaluating BEST STANDARD Model ---
  > FD Scores (Norm) - Spectral: 0.0552, Time: 5.4824

--- Evaluating FINAL SWA Model ---
  > FD Scores (Norm) - Spectral: 0.0894, Time: 5.8203

--- Generating Final Plots from the Winning Model ---
Best Standard Model is the winner!
Generated distribution plots (Fig 3a, 3b).
Generated per-channel PSD plots (Fig 4).
Generated connectivity plots (Fig 5).

All final evaluation plots have been saved to the output directory.


In [8]:
# ==============================================================================
# Cell 8: Downstream Task - Critic as Feature Extractor
# ==============================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np
import os

# --- 1. Define the Simple Classifier ---
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_classes=1):
        super(SimpleMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.layers(x)

# --- 2. Feature Extraction Function ---
def extract_critic_features(critic, data_loader, device):
    critic.eval()
    all_features = []
    with torch.no_grad():
        for eegs, _ in tqdm(data_loader, desc="Extracting Critic Features"):
            eegs = eegs.to(device)              # Shape: (B, T, C)
            x = eegs.transpose(1, 2)            # Shape: (B, C, T)
            features = critic.conv_layers(x)    # Just conv layers, no output head
            features = features.reshape(features.size(0), -1)
            all_features.append(features.cpu().numpy())
    return np.concatenate(all_features)

# --- 3. Main Classification Pipeline ---
print("\n--- Running Downstream Classification Task ---")

# Paths and config
OUTPUT_DIR = "/kaggle/working/"
PREPROCESSED_CLASSIFICATION_FILE = os.path.join('/kaggle/input/eeg-ad-male-female-classification-data', "classification_data_full.npz")
NUM_CLASSES = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load classification data
with np.load(PREPROCESSED_CLASSIFICATION_FILE) as data:
    X, y = data['X'], data['y']
    channel_indices = [0, 1, 2, 3, 4, 5, 6, 7]  # assuming order: F1, F2, C1, C2, P1, P2, O1, O2
    X = X[:, :, channel_indices]
# Update TIMESTEPS and CHANNELS dynamically
TIMESTEPS = X.shape[1]
CHANNELS = X.shape[2]

# Load the discriminator
discriminator = ImprovedDiscriminator(num_classes=NUM_CLASSES, channels=CHANNELS, timesteps=TIMESTEPS).to(device)
discriminator.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'discriminator_best.pth')))
discriminator.eval()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# --- Task A: Critic Features ---
X_train_feat = extract_critic_features(discriminator, train_loader, device)
X_test_feat = extract_critic_features(discriminator, test_loader, device)

train_feat_dataset = TensorDataset(torch.from_numpy(X_train_feat), torch.from_numpy(y_train).float())
test_feat_dataset = TensorDataset(torch.from_numpy(X_test_feat), torch.from_numpy(y_test).float())
train_feat_loader = DataLoader(train_feat_dataset, batch_size=128, shuffle=True)
test_feat_loader = DataLoader(test_feat_dataset, batch_size=128, shuffle=False)

# Classifier
model_critic = SimpleMLP(input_dim=X_train_feat.shape[1]).to(device)
optimizer = optim.Adam(model_critic.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

# Training loop
for epoch in range(50):
    model_critic.train()
    for features, labels in train_feat_loader:
        features, labels = features.to(device), labels.to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model_critic(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation
model_critic.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_feat_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model_critic(features)
        preds = (torch.sigmoid(outputs) > 0.5).float().cpu().numpy().flatten()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
accuracy_critic = accuracy_score(all_labels, all_preds)
print(f"  > Final Accuracy using Critic Features: {accuracy_critic:.4f}")



--- Running Downstream Classification Task ---


Extracting Critic Features: 100%|██████████| 22/22 [00:00<00:00, 123.35it/s]


  > Final Accuracy using Critic Features: 0.5039


In [9]:
# ==============================================================================
# Cell 9: Advanced Visualizations to Elevate Your Paper
# ==============================================================================
# This cell creates a latent space interpolation video, a powerful way to
# show that the generator has learned a smooth and meaningful manifold.
# ==============================================================================

def create_latent_space_interpolation_video(generator, save_path, n_steps=120):
    """Generates an animation of smoothly interpolating between two random points in the latent space."""
    generator.eval()
    
    # Define two random start/end points in the latent space
    z_start = torch.randn(1, LATENT_DIM, device=device)
    z_end = torch.randn(1, LATENT_DIM, device=device)
    # Use the same label for smooth transition
    label = torch.randint(0, NUM_CLASSES, (1,), device=device)
    
    fig, axes = plt.subplots(4, 2, figsize=(12, 10), sharex=True)
    axes = axes.flatten()
    lines = [ax.plot([], [], lw=2)[0] for ax in axes]
    channel_names = ['F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'O1', 'O2']

    def init():
        for i, ax in enumerate(axes):
            ax.set_ylim(-2, 2)
            ax.set_xlim(0, TIMESTEPS)
            ax.set_title(channel_names[i])
            ax.grid(True)
        return lines

    def update(frame):
        # Linear interpolation
        alpha = frame / n_steps
        z_interp = (1 - alpha) * z_start + alpha * z_end
        
        with torch.no_grad():
            generated_signal = generator(z_interp, label).cpu().numpy().squeeze()
        
        for i, line in enumerate(lines):
            line.set_data(np.arange(TIMESTEPS), generated_signal[:, i])
        fig.suptitle(f'Latent Space Interpolation (Frame {frame}/{n_steps})', fontsize=16)
        return lines
    
    ani = animation.FuncAnimation(fig, update, frames=n_steps, init_func=init, blit=True)
    # Save as a video file
    writer = animation.FFMpegWriter(fps=30)
    ani.save(save_path, writer=writer)
    plt.close()
    print(f"Latent space interpolation video saved to {save_path}")
    # To display in Kaggle, you might need to convert to GIF or embed HTML
    return save_path

# --- Generate the video using the best model ---
print("\n--- Generating Advanced Visualizations ---")
try:
    best_generator = ResGenerator(LATENT_DIM, NUM_CLASSES, CHANNELS, TIMESTEPS).to(device)
    best_generator.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'generator_best.pth')))
    create_latent_space_interpolation_video(best_generator, os.path.join(OUTPUT_DIR, "latent_space_interpolation.mp4"))
except FileNotFoundError:
    print("Best generator not found, skipping advanced visualization.")
except Exception as e:
    print(f"Could not create video. You might need to install ffmpeg: !conda install -c conda-forge ffmpeg -y. Error: {e}")


--- Generating Advanced Visualizations ---
Latent space interpolation video saved to /kaggle/working/latent_space_interpolation.mp4
